In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ANSH\\Desktop\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ANSH\\Desktop\\Chicken-Disease-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksconfig:
    root_dir: Path
    tensorboard_log_dir: Path
    checkpoint_model_filepath: Path




In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories


In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAM_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_prepare_callback_config(self) -> PrepareCallbacksconfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath) #getting only directory path
        create_directories([Path(model_ckpt_dir),
                            Path(config.tensorboard_root_log_dir)
                            ])

        prepare_callbacks_config =PrepareCallbacksconfig(
            root_dir = Path(config.root_dir),
            tensorboard_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath)
        )
        return prepare_callbacks_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksconfig):
        self.config=config

    @property
    def _create_tb_callbacks(self):
        timestamp=time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir= os.path.join(
            self.config.tensorboard_log_dir,
            f"tb_logs_at{timestamp}", 
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    """
    It creates TensorBoard callback object
    Which will log all training progress to that folder, 
    and TensorBoard will pick it up to display graphs and plots.
    """
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    #It will save the model to checkpoint_model_filepath only when validation loss improves (save_best_only=True).
    

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [10]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks= PrepareCallback(config= prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e